In [1]:
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

In [2]:
def inf_ing(v_ing):
    inf = v_ing
    if v_ing == 'broasting':
        inf = 'broast'
    elif v_ing == 'juicing':
        inf = 'juice'
    elif v_ing == 'parbaking':
        inf = 'parbake'
    elif v_ing == 'sautéing':
        inf = 'sauté'
    elif v_ing == 'charbroiling':
        inf = 'charbroil'
    elif v_ing == 'crinkle-cutting':
        inf = 'crinkle-cut'
    elif v_ing == 'cheesemaking':
        inf = 'cheesemake'
    elif v_ing == 'swissing':
        inf = 'swiss'
    elif v_ing == 'flash-frying':
        inf = 'flash-fry'
    elif v_ing == 'flashbaking':
        inf = 'flashbake'
    else:
        inf = WordNetLemmatizer().lemmatize(v_ing.lower(), 'v')
    return inf

def infinitive(words):
    aux = []
    word_list = words.split()
    for word in word_list:
        if word != 'cook' and word != 'cooking':
            if word.endswith('ing'):
                inf = inf_ing(word.lower())
                aux.append(inf)
                if inf == word:
                    print('NO INFINITIVE FOR', word)
            else:
                aux.append(word.lower())
    result = ' '.join(aux)
    return result

def trim(s):
    return ' '.join(s.split())

In [3]:
techniques = set()

In [4]:
def clean_1(l):
    r = []
    for e in l:
        if e not in [
            'Template:Cooking techniques',
            'List of cooking techniques',
            'List of twice-baked foods',
        ]:
            x = e.strip().lower().replace('_', ' ')
            x = x.split(' (')[0]
            x = infinitive(x)
            r.append(x)
    return r

In [5]:
with open('data/techniques/wikipedia_category_cooking_techniques.html') as f:
    soup = BeautifulSoup(f.read(), 'html.parser')
    elems = soup.select('div#mw-pages div.mw-category-group ul li a')
    techs = map(lambda x: trim(x['title']), elems)
    clean_techs = clean_1(techs)
    techniques = techniques.union(clean_techs)

In [10]:
def clean_4(l):
    r = []
    for e in l:
        if e not in [
            'Chinese cooking techniques',
            'Manifold Destiny',
            'Meat cooking techniques',
            'Vietnamese cooking techniques',
        ]:
            x = e.strip().lower().replace('_', ' ')
            x = x.split(' (')[0]
            x = infinitive(x)
            r.append(x)
    return r

In [11]:
with open('data/techniques/wikipedia_list_cooking_techniques.html') as f:
    soup = BeautifulSoup(f.read(), 'html.parser')
    elems = soup.select('div#bodyContent h3 ~ ul:nth-of-type(1) li > a:nth-of-type(1)')
    techs = map(lambda x: trim(x.string), elems)
    clean_techs = clean_4(techs)
    techniques = techniques.union(clean_techs)

In [18]:
len(techniques)

225

In [19]:
with open('data/techniques/wikipedia_techniques.txt', 'w') as f:
    f.write('\n'.join(sorted(techniques)))